In [ ]:
# 最好首先对数据进行可视化，这样分为多少个类心里有个印象
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

# 自己创建数据集
X, y = make_blobs(n_samples=500, n_features=2, centers=4, random_state=1)
# 生成1个子图，fig是画布，ax1是对象
fig, ax1 = plt.subplots(1)
ax1.scatter(X[:, 0], X[:, 1]
            ,marker="o" # 点的形状
            ,s=8 # 点的大小
            )
plt.show()

In [ ]:
from sklearn.cluster import KMeans
n_clusters = 3

cluster = KMeans(n_clusters=n_clusters, random_state=0).fit(X)

# 重要的属性labels_, 查看聚类好的类别，每个样本所对应的类
y_pred = cluster.labels_
y_pred
# kmeams并不需要建立模型或者预测结果，因此我们只需要fit就能得到聚类结果了
# 也有predict表示学习数据x并对x的类进行预测,但是和我们的labels属性结果相同
pre = cluster.fit_predict(X)
pre == y_pred
# 当数据量很大时，就需要predict，可以先用切片，切少量的样本，用fit找到质心，之后再将所有数据带入跑，就会比较好,比如很多万行
cluster_smallsub = KMeans(n_clusters=n_clusters, random_state=0).fit(X[:200])
y_pred_ = cluster_smallsub.predict(X)
# 查看质心
centroid = cluster.cluster_centers_
centroid

# 查看总距离的平方和
inertia = cluster.inertia_
inertia


In [ ]:
color = ["red", "pink", "orange", "gray"]
fig, ax1 = plt.subplots(1)
for i in range(n_clusters):
    ax1.scatter(X[y_pred==i, 0], X[y_pred==i, 1]
                ,marker="o"
                ,s=8
                ,c=color[i])
ax1.scatter(centroid[:, 0], centroid[:, 1]
            ,marker="x"
            ,s=15
            ,c="black")
plt.show()
# 接下来可以改变n_cluster，来看看，发现平方和不是评价指标，毕竟取500 的平方和是0



In [ ]:
# 如何评价聚类呢，无法确定，只能是根据业务来看，inartia没有界，对模型本身是好不好，而且本身容易受到特征数目的影响，会陷入维度诅咒，会受到超参数k的影响，而且假设是个凸分布
# 使用轮廓系数
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
# 把数据和对应的预测类别带入进行分析
silhouette_score(X, y_pred)
# 平均的样本轮廓系数
silhouette_score(X, cluster.labels_)
# 每个样本自己的轮廓系数
silhouette_samples(X, y_pred)


In [ ]:
#  另一个评价指数
from sklearn.metrics import calinski_harabaz_score
calinski_harabaz_score(X, y_pred)
# 为了计算速度，有%%timeit 魔法方法
# 这里我们用另一种计算时间的方法，时间戳计算运行速度
from time import time
# time(),记下每一次time（）命令的时间戳
t0 = time()
calinski_harabaz_score(X, y_pred)
time() - t0

t0 = time()
silhouette_score(X, y_pred)
time() - t0
# 时间戳可以通过datatime中的函数转换为真正的时间格式
import datetime
datetime.datetime.fromtimestamp(t0).strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm #colormap
import numpy as np
import pandas as pd


In [ ]:
# 基于我们的轮廓曲线选择最佳的n_clusters
# 知道每个聚出来的类的轮廓系数是多少，还想要一个各个类之间的轮廓系数的对比
# 知道聚类完毕之后的图像的分布是什么摸样
# fig 是画布生成的画布对象，ax1是图像

n_clusters = 4
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_size_inches(18, 7)
# 第一个图是我们的轮廓系数图像，是由各个蹙的轮廓系数组成的横向条形图、
# 横坐标是轮廓系数的取值， 纵坐标是我们的每个样本
# 首先设定纵坐标，轮廓系数在-1， 1之间，但我们希望是大于零de
ax1.set_xlim([-0.1, 1])
# 我们希望不同的簇之间有一定的空间，以便我们看到不同的条形图聚合成的块，理解她是对应了哪一个簇，因此多加了(n_clusters + 1) * 10
ax1.set_ylim([0, X.shape[0] + (n_clusters + 1) * 10])
# 开始建模，调用聚类好的标签
clusterer = KMeans(n_clusters=n_clusters, random_state=10).fit(X)
cluster_labels = clusterer.labels_
# 生成所有样本的轮廓系数的均值
silhouette_avg = silhouette_score(X, cluster_labels)
print("for n_clusters = ", n_clusters,
        "the average silhouette score is", silhouette_avg)
# 调用silhouette_samples,返回每个样本点的轮廓系数，知识我们的横坐标
sample_silhouette_values = silhouette_samples(X, cluster_labels)
# 开始画图
# 设定y轴上的初始值
y_lower = 10
# 对每个簇进行循环
for i in range(n_clusters):
    # 从每个样本的轮廓系数结果中抽取第i个簇的轮廓系数，并对他进行排序
    ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
    # 注意.sort()会直接改掉原数据的顺序, 令他是从小到大的顺序
    ith_cluster_silhouette_values.sort()
    # 查看这个簇中有多少样本
    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    # 这个簇在y轴上的取值
    y_upper = y_lower + size_cluster_i
    #colormap库中的，使用小数来调节颜色的函数
    # nipy_spectrol(输入任意一个小数代表颜色)
    # 我们希望每个簇的颜色不一样，我们需要的颜色种类刚好是循环的个数的种类，在这里，我们只要保证每次循环生成的小数是不同的，可以使用任意方式来获取小数
    # 我是用i的浮点数除以n_clusters，在不同的i下，自然生成不同的小数
    # 以确保不同的簇有不同的形状

    color = cm.nipy_spectral(float(i) / n_clusters)

    # 开始补充一图中的内容，fill_between是让一个范围中的柱状图都统一颜色的函数
    # fill_betweenx的范围是在纵坐标上
    # fill_betweeny的范围实在横坐标上
    # fill_betweenx的参数应该输入（纵坐标的下线， 纵坐标的上限，x轴上的取值， 柱状图的颜色）
    ax1.fill_betweenx(np.arange(y_lower, y_upper)
                        ,ith_cluster_silhouette_values
                        ,facecolor=color
                        ,alpha=0.7 # 透明度
    )
    # 为灭个簇的轮廓系数写上簇的编号，并且让粗的编号显示坐标轴上每个条形图的中间位置
    # text的参数为（要显示编号的位置的横坐标，要显示编号的位置的纵坐标，要显示的编号内容）
    ax1.text(-.05
            ,y_lower + 0.5 * size_cluster_i
            ,str(i))
    # 为下一个簇计算新的y轴上的初始值，是每一次迭代之后，y的上限再加上10
    # 以此来保证，不同的簇的图像之间显示有空隙
    y_lower = y_upper + 10

# 给图1加上
ax1.set_title("the silhouette plot for the various clusters")
ax1.set_xlabel("the silhouette coefficient values")
ax1.set_ylabel("cluster label")
# 把整个数据集中的轮廓系数的均值以虚线的形式放入我们的图中
ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
# 让y轴不显示任何刻度
ax1.set_yticks([])
# 让x轴上的刻度显示为我们规定的列表
ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

# 开始对第二个图进行操作,画出聚类的样子， 首先获取颜色，由于这里没有循环，因此我们需要一次性生成多个小数来获取多个颜色
# astype转化数据为浮点数，0， 1， 2， 3里面总共有四种颜色
colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
ax2.scatter(X[:, 0], X[:, 1]
            ,marker="o" # 点的形状
            ,s=8 # 点的大小 
            ,c=colors
            )
# 把生成的质心放到图像里去
centers = cluster.cluster_centers_
# draw white circles at cluster centers
ax2.scatter(centers[:, 0], centers[:, 1], marker="x", 
            c="red", alpha=1, s=200)
# 为图二设置标题
ax2.set_title("the visualization of the clustered data")
ax2.set_xlabel("feature space for the 1st feature")
ax2.set_ylabel("feature space for the 2nd feature")

# 为整个图设置标题
plt.suptitle("silhouette analysis for kmeans clustering on sample data"
            "with n_cluster = %d" % n_clusters,
            fontsize=14, fontweight="bold")
plt.show()



# 之后把上面的代码包装成为一个大循环，设置n_cluster 为2， 3，4 5，6再循环


In [ ]:
#看一看random，对于选择初始质心的影响

plus = KMeans(n_clusters=10).fit(X)
plus.n_iter_
# 10，迭代次数少
random = KMeans(n_clusters = 10, init="random",random_state=420).fit(X)
random.n_iter_
# 19，迭代次数多


#有k_means函数
from sklearn.cluster import k_means
k_means(X, 1, return_n_iter=True)
# return_n_iter 默认是false,可以查看返回的最佳迭代次数



In [ ]:
# 案例，聚类算法用于降维，kmeans的矢量量化,非结构数据是图像声音等等，是一种降维算法
# 矢量量化实在同等样本量上压缩信息大小
# 一张图片上的信息被聚类到了几个质心附近，然后利用质心的性质去代替这个簇里面的所有样本的性质
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
# 对两个序列中的点进行距离匹配的函数
from sklearn.datasets import load_sample_image
# 导入图片数据所用的类
from sklearn.utils import shuffle # 洗牌
# 打乱有序的东西



In [ ]:
china = load_sample_image("china.jpg") # 三维数据

In [ ]:
china.shape # 427*640里面的rgb三个特征
china[0][0]
newimage = china.reshape((427 * 640, 3))

In [ ]:
import pandas as pd
pd.DataFrame(newimage).drop_duplicates().shape
# 去掉重复值，总共有9万多个颜色


In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(china)
# 看图片长什么样子， 以上步骤都是处理图像必备的步骤

In [ ]:
# 再插入一张
flower = load_sample_image("flower.jpg")
plt.figure(figsize=(15, 15))
plt.imshow(flower)

In [ ]:
# 可以使用kmeans把9万多个颜色聚类到64个颜色，不损失太多的图片信息，就是把64个质心替换簇内的其他样本
n_clusters = 64
# plt.imshow再浮点数上表现非常优异，在这里我们把浮点数压缩到0 1 之间
china = np.array(china, dtype=np.float64) / china.max()
# 把china 从图片格式转换为矩阵格式，把三维数组转换为二维矩阵
# 把长宽高都先取出来，之后输入时只能输入二维数组，但是最后在imshow还是要以后用三维输出，是一种存储
w, h, d = original_shape = tuple(china.shape)
assert d == 3, "一个格子中的特征数不等于3"
# assert 相当于raise error if not ，表示为“不为true就报错”
# 要求d必须为三，否则就报错
# np.reshape(a, newshape, order="C"), reshape函数的第一个参数a是要改变结构的对象，第二个参数是要改变的新结构
image_array = np.reshape(china, (w * h, d))# reshape改变结构

In [ ]:

image_array
image_array.shape
# 无论有几维，只要相乘后的总数量不变，维度就可以随意变化

In [ ]:
# 接下来进行建模
# 收先先使用1000个数据找到初始质心
image_array_sample = shuffle(image_array, random_state=0)[:1000]
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(image_array_sample)
kmeans.cluster_centers_
# 选出来64个质心
# 选出质心之后，按照已经存在的质心对所有数据进行聚类
labels = kmeans.predict(image_array)
labels.shape# (273280,)

In [ ]:
set(labels)
# 使用质心来替换所有的样本,并且不破坏原有的数据
image_kmeans = image_array.copy()
image_kmeans # 27w 个样本点，9w多种不同的颜色（像素点）
labels # 这27万个样本点对应的额簇的质心的索引
#kmeans.cluster_centers_[labels[0]]
# 把对应的质心都取出来，到一个数组里
for i in range(w*h):
    image_kmeans[i] = kmeans.cluster_centers_[labels[i]]
# 查看生成的新图片信息
image_kmeans.shape# 273280, 3
pd.DataFrame(image_kmeans).drop_duplicates().shape
# 64, 3
# 恢复图片结构
image_kmeans = image_kmeans.reshape(w, h, d)
image_kmeans.shape

In [ ]:
# 随机进行的矢量量化
centroid_random = shuffle(image_array, random_state=0)[:n_clusters]# 从中随机抽出64个数字作为质心
labels_random = pairwise_distances_argmin(centroid_random,image_array,axis=0)
# x1和x2分别是序列
#用来计算x2中的每个样本点的距离，并返回和x2相同形状的，x1 中对应的最近的样本点的索引
# 就是对每个点都计算和64个质心的距离，返回最近的质心的索引,代表这27万对应的随机质心是哪一个
labels_random
len(set(labels_random))
#使用随机质心代替所有样本
image_random = image_array.copy()
for i in range(w * h):
    image_array[i] = centroid_random[labels_random[i]]
# 恢复图片形状
image_random = image_random.reshape(w, h, d)
image_random.shape

In [ ]:
# 看两种方式的区别
plt.figure(figsize=(10, 10))
plt.axis("off")
plt.title("original image,9w颜色")
plt.imshow(china)
plt.figure(figsize=(10, 10))
plt.axis("off")
plt.title("kmeans")
plt.imshow(image_kmeans)
plt.figure(figsize=(10, 10))
plt.axis("off")
plt.title("random")
plt.imshow(image_random)
plt.show()













